In [1]:
!pip install xlrd==2.0.0
# !pip install https://github.com/explosion/spacy-models/releases/download/de_core_news_lg-3.3.0/de_core_news_lg-3.3.0.tar.gz &> /dev/null

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sklearn.metrics
import matplotlib.pyplot as plt
import random
from random import sample
import nltk
from nltk.corpus import stopwords
from sklearn.base import TransformerMixin
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import roc_auc_score, accuracy_score
import glob
from sklearn.feature_selection import SelectPercentile, f_classif, SelectKBest, chi2
from sklearn.pipeline import Pipeline
from time import time
from sklearn.linear_model import LogisticRegression
from scipy.stats import spearmanr

rng = np.random.default_rng(4711)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
commentsDF = pd.DataFrame(columns={'ID'})
for file in glob.glob("/content/drive/MyDrive/MA/Daten/*"):
     tempDF = pd.read_excel(file)
     commentsDF = pd.concat([commentsDF, tempDF], join='outer')

commentsDF = commentsDF.drop(columns=['ID', 'Active', 'CID', 'Sperrhinweis', 'Loginname'],axis=1)

commentsDF['Date'] =  pd.to_datetime(commentsDF['Date'], format='%Y-%m-%d %H:%M:%S')
commentsDF['Time'] = pd.to_datetime(commentsDF['Date']).dt.time
commentsDF['Date'] = pd.to_datetime(commentsDF['Date']).dt.date
commentsDF['Date'] = pd.to_datetime(commentsDF['Date'], format='%Y-%m-%d')


commentsDF['Text'] = commentsDF['Text'].astype(str)
commentsDF['Title'] = commentsDF['Title'].astype(str)

commentsDF = commentsDF.drop_duplicates(subset=['Text']) 
commentsDF = commentsDF.sort_values(by=['Date']).reset_index(drop=True)

In [25]:
german_stop_words_import = open ("/content/drive/MyDrive/MA/stopwords-de.csv", "r", encoding="utf-8")
german_stop_words = pd.read_csv(german_stop_words_import, names=['stopwords'])['stopwords'].to_list()


vectorizer = TfidfVectorizer(stop_words=german_stop_words, 
                             ngram_range=(1,2), min_df = 3, max_features=4000)
vectorizer = vectorizer.fit(commentsDF.Text)
# tfidf_pickle = open ("/content/drive/MyDrive/MA/preprocessor_RP_Comments_noDupes.pk", "rb")
# preprocessed = pickle.load(tfidf_pickle)

# tfidf = vectorizer.fit_transform(tfidf_load)
# tfidf_reduced = SelectKBest(k=4000).fit_transform(tfidf, y_vals).toarray()

In [28]:
def get_features(comments):
    X = vectorizer.transform(comments)
    freq_array = X.toarray()
    # print(freq_array)
    freq_array_sum = np.sum(freq_array, axis=0)

    return freq_array_sum

def calculate_spear(corpus1, corpus2):
    features1 = get_features(corpus1)
    features2 = get_features(corpus2)
    # print('Length of feature arr: ',len(features1),len(features2))

    spear_results = spearmanr(features1, features2)

    return spear_results

def make_10_batches(inputDF):
  startDate = inputDF.sort_values(by='Date').iloc[0].Date
  endDate = inputDF.sort_values(by='Date').iloc[len(inputDF)-1].Date
  batchLength = (endDate - startDate).days / 10

  batchDF = pd.DataFrame(index=np.arange(22016))

  for i in range(1,11):
   batchStart = startDate + timedelta(days=((i-1) * batchLength))
   batchEnd = startDate + timedelta(days=(i * batchLength))
   mask = (inputDF.Date >= batchStart) & (inputDF.Date < batchEnd)
   batchDF[i] = pd.Series(inputDF.Text[mask].dropna()).reset_index(drop=True)
  
  return batchDF.dropna()

def calc_srcc_matrix(input): 
  batchDF = make_10_batches(input)
  srcc_matrix = pd.DataFrame(index=np.arange(1,11), columns=np.arange(1,11))

  for i in range(1,11):
    for j in range(1,11):
      srcc_matrix.iloc[i-1,j-1] = calculate_spear(batchDF[i], batchDF[j])[0]

  return srcc_matrix


In [29]:
srcc_matrix = calc_srcc_matrix(commentsDF)
srcc_matrix

,1,2,3,4,5,6,7,8,9,10
1,1.0,0.840968,0.804369,0.799098,0.756357,0.751613,0.770206,0.749472,0.614303,0.65339
2,0.840968,1.0,0.827628,0.808329,0.771685,0.772101,0.78416,0.764543,0.630183,0.679939
3,0.804369,0.827628,1.0,0.838712,0.785288,0.795464,0.778538,0.74908,0.634837,0.671159
4,0.799098,0.808329,0.838712,1.0,0.799131,0.809748,0.792093,0.769402,0.626711,0.683521
5,0.756357,0.771685,0.785288,0.799131,1.0,0.813293,0.774123,0.756689,0.64122,0.687314
6,0.751613,0.772101,0.795464,0.809748,0.813293,1.0,0.843694,0.786685,0.620972,0.67644
7,0.770206,0.78416,0.778538,0.792093,0.774123,0.843694,1.0,0.802643,0.634589,0.680845
8,0.749472,0.764543,0.74908,0.769402,0.756689,0.786685,0.802643,1.0,0.665338,0.707091
9,0.614303,0.630183,0.634837,0.626711,0.64122,0.620972,0.634589,0.665338,1.0,0.821117
10,0.65339,0.679939,0.671159,0.683521,0.687314,0.67644,0.680845,0.707091,0.821117,1.0


In [38]:
inputDF = commentsDF
startDate = inputDF.sort_values(by='Date').iloc[0].Date
endDate = inputDF.sort_values(by='Date').iloc[len(inputDF)-1].Date
batchLength = (endDate - startDate).days / 10

batchDF = pd.DataFrame(index=np.arange(22016))

for i in range(1,11):
  batchStart = startDate + timedelta(days=((i-1) * batchLength))
  batchEnd = startDate + timedelta(days=(i * batchLength))
  print(batchStart, batchEnd)

batchDF

2018-11-01 00:00:00 2018-12-31 14:24:00
2018-12-31 14:24:00 2019-03-02 04:48:00
2019-03-02 04:48:00 2019-05-01 19:12:00
2019-05-01 19:12:00 2019-07-01 09:36:00
2019-07-01 09:36:00 2019-08-31 00:00:00
2019-08-31 00:00:00 2019-10-30 14:24:00
2019-10-30 14:24:00 2019-12-30 04:48:00
2019-12-30 04:48:00 2020-02-28 19:12:00
2020-02-28 19:12:00 2020-04-29 09:36:00
2020-04-29 09:36:00 2020-06-29 00:00:00


""
0
1
2
3
4
...
22011
22012
22013
22014


In [30]:
srcc_matrix.to_excel('SRCC Matrix.xlsx')